$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}$

# Assignment 5: Neural Networks

## Matt Gorbett

## Overview

This was a fun assignment.  I had a good time trying to find the best network structures possible without spending too much time.  The larger network was better for regression, but it took a long time.  I implemented my iteration count by seeing when the larger model structure started to overfit.  I'll have a look at the source code in the included files later this week to get a better feel for this neural network code.  

## Main Functions

### trainNNs
ml.partition is a simple function to break out the arrays for training inputs/outputs and testing inputs/outputs.  This was convenient.  Next, I want to iterate through each hidden layer I input into the function, which I do with a simple for loop.  I start taking time, because we want to log how many times it takes the network to run N number of repetitions on a given structure, per the requirements.  
Next, I have another for loop that tells us to run the neural network structure N number of times.  This will give us an idea of how the network performs generally, rather than just once.  Since training can give varying weights to a neural networks nodes, we want to gain a general idea of how this structure performs.  
The only difference between regression and classification is which constructor you use.  This is handled in the next lines of code with either using NeuralNetworkClassifier or NeuralNetwork constructor.  
Next I train the network with the training data and predict the test data.  I use SkLearn's mean_squared_error to obtain RMSE results.  I append these to separate lists for train and test data and continue the loops, logging time and results where applicable.  

### summarize
summarize gets an average of the N different runs of each network.  This just involves iterating through each network structures' results and taking the average RMSE result of each train and test sets predictions.  np.average worked well here.  I then added the structure and duration along with these averages into a new list and returned.  

### bestNetwork
This function finds the network structure in the summary results that has the best test RMSE.  This is determined by finding the network with the lowest error value, the third element of the summary array.  

In [29]:
import numpy as np
import pandas as pd
import neuralnetworks as nn
import mlutils as ml 
from sklearn.metrics import mean_squared_error
from math import sqrt
import time
import matplotlib.pyplot as plt

#global errors
#errors = []

def trainNNs(X, T, trainFraction, hiddenLayerStructures, numberRepetitions, numberIterations, classify=False):
    results=[]
    Xtrain,Ttrain,Xtest,Ttest=ml.partition(X,T,[trainFraction, 1-trainFraction], classification=classify)
    i=0
    for hiddenLayer in hiddenLayerStructures:
        i+=1
        trainList=[]
        testList=[]
        now = time.time()
        for repetition in range(0,numberRepetitions):
            if(classify):
                classifiers=int(T.max(axis=0))+1
                nnet = nn.NeuralNetworkClassifier(X.shape[1], hiddenLayer, classifiers)
            else:     
                nnet = nn.NeuralNetwork(X.shape[1], hiddenLayer, T.shape[1])
            
            nnet.train(Xtrain, Ttrain, numberIterations)
            trainPredict=nnet.use(Xtrain)
            testPredict=nnet.use(Xtest)
            rmseTrain = sqrt(mean_squared_error(Ttrain, trainPredict))
            rmseTest=sqrt(mean_squared_error(Ttest, testPredict))
            print("Network #"+ str(i)+" repetition #"+ str(repetition+1)+ " has test RMSE="+str(rmseTest))
            trainList.append(rmseTrain)
            testList.append(rmseTest)
            
        later = time.time()
        duration=later-now
        errors.append([hiddenLayer, nnet.getErrorTrace()])
        results.append([hiddenLayer, trainList, testList, duration])
    return results

def summarize(results):
    summary=[]
    for x in results:
        trainAvg=np.average(x[1])
        testAvg=np.average(x[2])
        summary.append([x[0],trainAvg,testAvg,x[3]])
    return summary
        
    
def bestNetwork(summary):
    lowest=summary[0][2]
    lowestX=summary[0]
    for x in summary:
        if(x[2]<lowest):
            lowest=x[2]
            lowestX=x
    return lowestX

## Regression Experiment





I split this experiment into a few different sections:
1.  Load data and experiment with optimal number of iterations.  
2.  Run main experiment and print results.
3.  Train optimal network structure with training set and predict test data.  Then graph results to see how I did.  
4.  Plot error trace for each network



To find the optimal number of iterations:
--  Train the largest network I plan on testing with the energy data.
--  Print error trace to see if training is still occuring.  Since error is still decreasing after 1000 iterations on train data, I want to test whether overfitting is occuring.
--  Test for overfitting on test data by checking the RMSE on train and test data for a list of iterations.  This will determine the ideal number of iterations to train with.  

Model started to overfit at 1000 iterations.  

IDEAL ITERATIONS=500

In [32]:
X=pd.read_csv('energydata_complete.csv').iloc[:,1:-2]

names=X.columns
Xnames=names[2:].tolist()
Tnames=names[:2].tolist()
data=X.values
Xenergy=data[:,2:]
Tenergy=data[:,:2]



#nnet.train(Xenergy, Tenergy, 1000)
np.set_printoptions(threshold=np.nan)
#print(nnet.getErrorTrace())


Xtrain,Ttrain,Xtest,Ttest=ml.partition(Xenergy,Tenergy,[.8, .2])

iterationsList=[100,200,300,400,500,1000]
for it in iterationsList:
    nnet = nn.NeuralNetwork(Xenergy.shape[1], [50,50], Tenergy.shape[1])
    nnet.train(Xtrain, Ttrain, it)
    testPredict=nnet.use(Xtest)
    rmseTest=sqrt(mean_squared_error(Ttest, testPredict))
    trainPredict=nnet.use(Xtrain)
    rmseTrain = sqrt(mean_squared_error(Ttrain, trainPredict))
    print("Iterations: "+str(it))
    print("RMSE on train data: " +str(rmseTrain))
    print("RMSE on test data: " +str(rmseTest))
    print("")


Iterations: 100
RMSE on train data: 62.04301580595364
RMSE on test data: 62.3219509548606

Iterations: 200
RMSE on train data: 55.45469579836838
RMSE on test data: 59.04164998276852

Iterations: 300
RMSE on train data: 48.68748914505726
RMSE on test data: 56.75187265348035

Iterations: 400
RMSE on train data: 44.792834860375756
RMSE on test data: 56.06867330155281

Iterations: 500
RMSE on train data: 40.104647380413574
RMSE on test data: 55.23181882463125

Iterations: 1000
RMSE on train data: 35.82967082636165
RMSE on test data: 56.70895028330729



## Experiment
Below I train and test on 10 different networks 3 times for 500 iterations each.  At the bottom I graph the error trace for each models last run.  

In [ ]:
print("\nTraining 10 neural networks 3 times each for 500 iterations")
results = trainNNs(Xenergy, Tenergy, .8, [0,1,10,20,50, 100, 500,[10,10], [20,20], [50,50]], 3, 500)


summary=summarize(results)
print("\nSummary:")
print(summary)
print("")

print("Best Network:")
best=bestNetwork(summarize(results))
print('Hidden Layers {} Average RMSE Training {:.2f} Testing {:.2f} Took {:.2f} seconds'.format(*best))

print("\nError trace for each model")
%matplotlib inline
for i in range(len(errors)):
    plt.plot(errors[i][1], label=errors[i][0])
    
plt.legend()
plt.show()

# Results
Below I train on the best network and print out the first 50 predictions compared to their actual results for both appliances and lights.  

In [ ]:
Xtrain,Ttrain,Xtest,Ttest=ml.partition(Xenergy,Tenergy,[.8,.2])
nnet = nn.NeuralNetwork(Xtrain.shape[1], best[0], Ttrain.shape[1])

nnet.train(Xtrain, Ttrain, 500)
testPredict=nnet.use(Xtest)

appliancesPredict = [item[0] for item in testPredict]
lightsPredict = [item[1] for item in testPredict]

appliancesActual = [item[0] for item in Ttest]
lightsActual = [item[1] for item in Ttest]




%matplotlib inline
plt.title("Appliances")
plt.plot(np.arange(len(appliancesPredict))[0:50], appliancesActual[0:50], 'o-', label='actual');
plt.plot(np.arange(len(appliancesPredict))[0:50], appliancesPredict[0:50], 'o-', label='prediction');
plt.legend(loc='upper left')


In [ ]:
%matplotlib inline
plt.title("Lights")
plt.plot(np.arange(len(lightsActual))[0:50], lightsActual[0:50], 'o-', label='actual');
plt.plot(np.arange(len(lightsPredict))[0:50], lightsPredict[0:50], 'o-', label='prediction');
plt.legend(loc='upper left')


## Classification Experiment

I split the cells up as follows:
1.  Load data, make sure the arrays are loaded properly.
2.  Find best number of iterations like above.
3.  Train, test, and graph the best model with a confusion matrix. 

Pandas has a function called "Categorical", which made it easy to convert each classification into an integer.  

In [ ]:

pd.options.display.precision = 4
Xanuran=pd.read_csv('Frogs_MFCCs.csv').iloc[:,1:22]
Y=pd.read_csv('Frogs_MFCCs.csv').iloc[:,24]
Y = pd.Categorical(Y)
Tanuran = Y.codes
Tanuran=Tanuran[:,np.newaxis]

Xanuran.shape, Tanuran.shape
Xanuran=Xanuran.values

classes=np.unique(Y)

for i in range(10):
    print('{} samples in class {}'.format(np.sum(Tanuran==i), i))

# Iterations

Below I do the same as above and try to get the best number of iterations to train without overfitting my biggest structure.  

Iterations=500

In [ ]:
Xtrain,Ttrain,Xtest,Ttest=ml.partition(Xanuran,Tanuran,[.8, .2])

iterationsList=[100,200,300,400,500,1000]
for it in iterationsList:
    nnet = nn.NeuralNetwork(Xanuran.shape[1], [50,50], Tanuran.shape[1])
    nnet.train(Xtrain, Ttrain, it)
    testPredict=nnet.use(Xtest)
    rmseTest=sqrt(mean_squared_error(Ttest, testPredict))
    trainPredict=nnet.use(Xtrain)
    rmseTrain = sqrt(mean_squared_error(Ttrain, trainPredict))
    print("Iterations: "+str(it))
    print("RMSE on train data: " +str(rmseTrain))
    print("RMSE on test data: " +str(rmseTest))
    print("")

##  Testing

The biggest thing here was implementing the classification.  I needed to get the number of categories to send to the neural network classifier.  I used RMSE as the error function, and plotted the results using a pandas confusion matrix.  Error trace is also plotted below.  

In [ ]:
errors=[]

#results = trainNNs(Xanuran, Tanuran, 0.8, [0, 5, [5, 5]], 5, 100, classify=True)
results = trainNNs(Xanuran, Tanuran, 0.8,[0,1,10,20,50, 100, 500,[10,10], [20,20], [50,50]], 3, 500, classify=True)

print("\nResults:")
print(results)

print("\nSummary:")
summary=summarize(results)
print(summary)
print("\nBest Network:")
best=bestNetwork(summarize(results))
print(best)



Xtrain,Ttrain,Xtest,Ttest=ml.partition(Xanuran, Tanuran,[.8,.2])

classifiers=int(Tanuran.max(axis=0))+1
nnet = nn.NeuralNetworkClassifier(Xtrain.shape[1], best[0], classifiers)

nnet.train(Xtrain,Ttrain, 500)
testPredict=nnet.use(Xtest)


testPredict = testPredict[:,0]
Ttest = Ttest[:,0]

#tried to use ml.confusionMatrix, but it looked kind of ugly.  Using pandas instead.  
#ml.confusionMatrix(Ttest,testPredict, [0,1,2,3,4,5,6,7,8,9])

print("\nConfusion Matrix of Actual/Predicted for test set of the best model")
y_actu = pd.Series(Ttest, name='Actual')
y_pred = pd.Series(testPredict, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred, margins=True)
print(df_confusion)

print("\nError trace for each model")
%matplotlib inline
for i in range(len(errors)):
    plt.plot(errors[i][1], label=errors[i][0])
    
plt.legend()
plt.show()

## Grading

In [25]:
%run -i "A5grader.py"


======================= Code Execution =======================


Testing summarize([[[1,1], [1.2, 1.3, 1.4], [2.2, 2.3, 2.4], 0.5], [[2,2,2], [4.4, 4.3, 4.2], [6.5, 6.4, 6.3], 0.6]])

--- 10/10 points. Correctly returned [[[1, 1], 1.3, 2.3000000000000003, 0.5], [[2, 2, 2], 4.3, 6.3999999999999995, 0.6]]

Testing bestNetwork([[[1, 1], 1.3, 2.3, 0.5], [[2, 2, 2], 4.3, 1.3, 0.6]])

--- 10/10 points. Correctly returned [[2, 2, 2], 4.3, 1.3, 0.6]

X = np.random.uniform(-1, 1, (100, 3))
T = np.hstack(((X**2 - 0.2*X**3).sum(axis=1,keepdims=True),
               (np.sin(X)).sum(axis=1,keepdims=True)))
result = trainNNs(X, T, 0.7, [0, 5, 10, [20, 20]], 10, 100, False)
Network #1 repetition #1 has test RMSE=0.33780054524861164
Network #1 repetition #2 has test RMSE=0.33780054524861164
Network #1 repetition #3 has test RMSE=0.33780054524861164
Network #1 repetition #4 has test RMSE=0.33780054524861164
Network #1 repetition #5 has test RMSE=0.33780054524861164
Network #1 repetition #6 has test RM

# Extra Credit

## Regression

For regression, I'm using the dataset http://archive.ics.uci.edu/ml/datasets/SGEMM+GPU+kernel+performance

This is a GPU performance test.  I will be using all inputs and only one output: Run1.  

I will only train and test on the first 50,000 rows because I'm having issues with the data rate on jupyter.  

In [ ]:
errors=[]
data=pd.read_csv('sgemm_product.csv').iloc[:50000,0:-3]


x=data.iloc[:,0:-1].values
y=data.iloc[:,14].values
y=y[:,np.newaxis]



print("\nTraining 10 neural networks 3 times each for 500 iterations")
results_ = trainNNs(x, y, .8, [0,1,10,20,50, 100, 500,[10,10], [20,20], [50,50]], 5, 500)


summary_=summarize(results_)
print("\nSummary:")
print(summary_)
print("")

print("Best Network:")
best_=bestNetwork(summarize(results))
print('Hidden Layers {} Average RMSE Training {:.2f} Testing {:.2f} Took {:.2f} seconds'.format(*best))

print("\nError trace for each model")
%matplotlib inline
for i in range(len(errors)):
    plt.plot(errors[i][1], label=errors[i][0])
    
plt.legend()
plt.show()

## Classification

For classification I'm using http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Coimbra
I'll be using all columns. 
This is a binary classification problem with a balanced dataset of cancer/non-cancer.  

In [ ]:
x=pd.read_csv('dataR2.csv').iloc[:,0:-1]
y_=pd.read_csv('dataR2.csv').iloc[:,-1]


y_ = pd.Categorical(y_)
y = y_.codes
y=y[:,np.newaxis]

x.shape, y.shape
x=x.values

classes=np.unique(y)

for i in range(2):
    print('{} samples in class {}'.format(np.sum(y==i), i))
    
    
results = trainNNs(x, y, 0.8,[0,1,10,20,50, 100, 500,[10,10], [20,20], [50,50]], 5, 2000, classify=True)

print("\nResults:")
print(results)

print("\nSummary:")
summary=summarize(results)
print(summary)
print("\nBest Network:")
best=bestNetwork(summarize(results))
print(best)



Xtrain,Ttrain,Xtest,Ttest=ml.partition(x, y,[.8,.2])

classifiers=int(y.max(axis=0))+1
nnet = nn.NeuralNetworkClassifier(Xtrain.shape[1], best[0], classifiers)

nnet.train(Xtrain,Ttrain, 2000)
testPredict=nnet.use(Xtest)


testPredict = testPredict[:,0]
Ttest = Ttest[:,0]

#tried to use ml.confusionMatrix, but it looked kind of ugly.  Using pandas instead.  
#ml.confusionMatrix(Ttest,testPredict, [0,1,2,3,4,5,6,7,8,9])

print("\nConfusion Matrix of Actual/Predicted for test set of the best model")
y_actu = pd.Series(Ttest, name='Actual')
y_pred = pd.Series(testPredict, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred, margins=True)
print(df_confusion)


# Confidence Intervals
I used code from the link to calculate the confidence interval for each model in the regression dataset.

In [ ]:
import scipy

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h


CI=[]
for r in results_:
    mean,low,high=mean_confidence_interval(r[2])
    CI.append([r[0],low,mean,high])

    
df_ci = pd.DataFrame(CI,columns=['Structure','Low','Mean','High'])
print(df_ci)

I ran each structure 5 times to get a better idea of how the model performs.  All models performed very close to each other for the regression dataset, which implies that the data is very trainable.  

The first structure had the lowest deviation between its low and high.  Since every model trained to the same RMSE, it was simple.  

The structure with the largest deviation between low and high was 100 hidden layers.  This still only deviated by a little over 1 RMSE.   